# Imports

In [ ]:
from IPython.utils import io
from IPython.display import display, Math, HTML

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import os
import numpy as np
import pandas as pd
from datetime import datetime
import dateutil
import time
import re
import geopy.distance
import difflib
from scipy.optimize import minimize
from functools import partial

import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)

import json
import numbers


class TicToc(object):
    def __init__(self):
        self.last_t = time.time()

    def tic(self):
        self.last_t = time.time()
        return

    def print_elapsed(self, dt):
        days = int(dt // (3600 * 24))
        hours = int((dt - days * 3600 * 24) // 3600)
        minutes = int((dt - days * 3600 * 24 - hours * 3600) // 60)
        seconds = dt - days * 3600 * 24 - hours * 3600 - minutes * 60

        s = "Elapsed "
        if days > 0:
            s += f"{days} day" + ("s, " if days != 1 else ", ")
        if hours > 0 or days > 0:
            s += f"{hours} hour" + ("s, " if hours != 1 else ", ")
        if minutes > 0 or hours > 0 or days > 0:
            s += f"{minutes} minute" + ("s and " if minutes != 1 else " and ")

        s += f"{seconds:1.2f} seconds"

        return s

    def toc(self, b_print=True):
        elapsed = time.time() - self.last_t
        if b_print:
            print(self.print_elapsed(elapsed))
        else:
            return elapsed


tictoc = TicToc()


class str_print(object):
    def __init__(self):
        self._s = ""

    def __call__(self, s=""):
        print(s)
        self._s += "\n" + s

    def get_str(self):
        return self._s


pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# Get a personal copy of the data of the author at:
# https://drive.google.com/file/d/1GfAB6PpC8voXshiZKADX6KU9FjjY8d7f/view?usp=sharing
# Should the link not work, please, drop me an email to guido.gigante@gmail.com
data_path = r"your_path/where_you_unzipped_the_data/Data"

# Utility functions

In [ ]:
# Utility functions
def signif_digit(x, n, as_string=False):
    x = np.array(x)
    x0 = x.copy()
    x0[x0 == 0.0] = 1.0
    k = np.floor(np.log10(np.abs(x0)))
    x_round = np.round(x * 10 ** (n - k - 1)) * 10 ** (k - n + 1)
    if as_string:
        k = np.array(k)
        x_round = np.array(x_round)
        if k.shape == ():
            k = np.array([k])
            x_round = np.array([x_round])

        x_round = [
            eval(f'f"{{{str(xr)}:1.{max(0, int(n - kx - 1))}f}}"')
            for kx, xr in zip(k, x_round)
        ]
        if x.shape == ():
            return x_round[0]
        else:
            return np.array(x_round)
    else:
        return x_round


##################################################################################
# Adam optimizer
def power_lr(step, lr0, scale=4 * 10**5, gamma=0.75):
    lr = lr0 / (1 + step / scale) ** gamma

    return lr


def _adam_1_step(
    params,
    train_data_it,
    f,
    v,
    sqr,
    v_bias_corr,
    sqr_bias_corr,
    lr_f=lambda step: power_lr(step=step, lr0=10**-3, scale=10**8, gamma=0.75),
    step=0,
    beta1_f=lambda step: 0.9,
    beta2_f=lambda step: 0.999,
    eps_stable=1e-8,
    mode="adam",
    clip_values=[-np.inf, np.inf],
):
    beta1 = beta1_f(step)
    if beta2_f is None:
        beta2 = None
    else:
        beta2 = beta2_f(step)

    if mode == "rmsprop":
        v *= beta1
        params -= lr_f(step) * v

    # get new batch
    batch, f_named_params = next(train_data_it)
    # compute gradient
    value, g_orig = f(params, batch, **f_named_params)
    g = np.clip(g_orig, a_min=clip_values[0], a_max=clip_values[1])

    if mode == "adam":
        v[:] = beta1 * v + (1.0 - beta1) * g

        if beta2 is not None and beta2 >= 0.0:
            sqr[:] = beta2 * sqr + (1.0 - beta2) * np.square(g)

            v_bias_corr[:] = v / (1.0 - beta1 ** (step + 1))
            sqr_bias_corr[:] = sqr / (1.0 - beta2 ** (step + 1))

            div = lr_f(step) * v_bias_corr / (np.sqrt(sqr_bias_corr) + eps_stable)
        else:  # Simple stochastic gradient descent
            div = lr_f(step) * v

        params -= div
    else:  # rmsprop
        if beta2 is not None and beta2 >= 0.0:
            sqr[:] = beta2 * sqr + (1.0 - beta2) * np.square(g)
            sqr_bias_corr[:] = sqr / (1.0 - beta2 ** (step + 1))
            # if ((step + 1) % 100) == 0:
            # print(np.sqrt(sqr_bias_corr) + eps_stable)
            g_norm = g / (np.sqrt(sqr_bias_corr) + eps_stable)
            v += g_norm
            params -= lr_f(step) * g_norm
        else:  # Simple stochastic gradient descent (with momentum)
            if np.any(sqr_bias_corr > 0.0):
                # g_norm = g / (np.sqrt(sqr_bias_corr) + eps_stable).max()
                g_norm = g / (np.sqrt(sqr_bias_corr) + eps_stable)
            else:
                g_norm = g
            v += g_norm
            params -= lr_f(step) * v

    return value, g_orig


def adam(
    params0,
    train_data_it,
    test_data_generator,
    f,
    n_steps=10**3,
    lr_f=lambda step: power_lr(step=step, lr0=10**-3, scale=10**8, gamma=0.75),
    beta1=0.9,
    beta2=0.999,
    eps_stable=1e-8,
    params_history_steps=None,
    delta_history_steps_min=10,
    mode="adam",
    stop_f=None,
    display_f=None,
    internal_data=None,
    clip_values=[-np.inf, np.inf],
):

    msg = ""

    values = np.zeros((n_steps,))
    grads = np.zeros((params0.size, n_steps))

    n_history_steps = None
    if params_history_steps is None:
        n_history_steps = 100
    elif type(params_history_steps) != np.ndarray:  # int, np.int32, np.int64...
        n_history_steps = params_history_steps

    if type(params_history_steps) != np.ndarray:
        params_history_steps = (
            np.unique(
                (
                    10 ** np.linspace(np.log10(1), np.log10(n_steps), n_history_steps)
                ).astype(int)
            )
            - 1
        )

        k0 = 0
        for k in range(1, params_history_steps.size):
            if (
                params_history_steps[k] - params_history_steps[k0]
                < delta_history_steps_min
            ):
                params_history_steps[k] = -1
            else:
                k0 = k
        params_history_steps = params_history_steps[params_history_steps != -1]
        del k0, k

    k_params_history_steps = 0
    params_history = {}
    test_error_history = {}

    params = params0.copy()
    v = np.zeros_like(params)
    sqr = np.zeros_like(params)
    v_bias_corr = np.zeros_like(params)
    sqr_bias_corr = np.zeros_like(params)

    if type(beta1) == float:
        beta1_f = lambda step: beta1
    else:
        beta1_f = beta1

    if type(beta2) == float:
        beta2_f = lambda step: beta2
    else:
        beta2_f = beta2

    for step in range(n_steps):
        if internal_data is not None:
            internal_data["params"] = params

        # print(f"Adam, step {step + 1}/{n_steps}")
        value, g = _adam_1_step(
            params,
            train_data_it,
            f,
            v,
            sqr,
            v_bias_corr,
            sqr_bias_corr,
            lr_f=lr_f,
            step=step,
            beta1_f=beta1_f,
            beta2_f=beta2_f,
            eps_stable=eps_stable,
            mode=mode,
            clip_values=clip_values,
        )
        values[step] = value
        grads[:, step] = g

        b_add_history = (
            k_params_history_steps < params_history_steps.size
            and params_history_steps[k_params_history_steps] == step
        )
        if b_add_history:
            params_history[step + 1] = params.copy()

            test_err = 0.0
            for td, f_named_params in test_data_generator():
                value, _ = f(params, td, **f_named_params)
                test_err += value

            test_error_history[step + 1] = test_err

            k_params_history_steps += 1

        if display_f is not None:
            display_data = {
                "step": step,
                "params": params,
                "values": values[: step + 1],
                "grads": grads[:, : step + 1],
                "params_history": params_history,
                "test_error_history": test_error_history,
                "b_add_history": b_add_history,
            }
            display_f(display_data)

        if stop_f is not None:
            stop_data = {
                "step": step,
                "params": params,
                "values": values[: step + 1],
                "grads": grads[:, : step + 1],
                "params_history": params_history,
                "test_error_history": test_error_history,
                "b_add_history": b_add_history,
            }
            b_stop = stop_f(stop_data)

            if b_stop:
                msg = "Minimization interrupted by stop function."
                break

    minimize_res = {
        "n_steps": step + 1,
        "params_history": params_history,
        "test_error_history": test_error_history,
        "values": values[: step + 1],
        "grads": grads[:, : step + 1],
        "message": msg,
        "params0": params0,
        "params": params,
    }

    return minimize_res


##################################################################################
# Functions to save and load json files
def _concat_cols(c):
    if type(c) == str:
        return c
    else:
        lst = [s for s in c if len(s) > 0]
        if len(lst) > 1:
            return "(" + ", ".join([s for s in c if len(s) > 0]) + ")"
        else:
            return lst[0]


class NPEncoder(json.JSONEncoder):
    def default(self, obj):
        # if isinstance(obj, np.ndarray):
        if callable(getattr(obj, "tolist", None)):
            return obj.tolist()
        elif isinstance(obj, jaxlib.xla_extension.DeviceArray):
            return obj.tolist()
        elif isinstance(obj, pd.core.frame.DataFrame):
            return {_concat_cols(c): obj[c].values for c in obj.columns}
        elif isinstance(
            obj,
            (
                np.int8,
                np.int16,
                np.int32,
                np.int64,
                np.uint8,
                np.uint16,
                np.uint32,
                np.uint64,
            ),
        ):
            return int(obj)
        elif isinstance(obj, (np.float32, np.float64)):
            return float(obj)

        return json.JSONEncoder.default(self, obj)


def check_array(l):
    b = isinstance(l, list) and len(l) > 0
    if b:
        b = np.all([isinstance(x, numbers.Number) for x in l])
    return b


def check_2d_matrix(l):
    b = isinstance(l, list) and len(l) > 0
    if b:
        b = np.all([check_array(sl) for sl in l])
    if b:
        lens = np.array([len(sl) for sl in l])
        b = np.all(lens == lens[0])
    return b


def np_obj_hook(o, s=None):
    if isinstance(o, dict):
        return {item[0]: np_obj_hook(item[1], item[0]) for item in o.items()}
    else:
        # if s is not None:
        # print([s, check_array(o), check_2d_matrix(o)])
        # # print([s, o])
        if check_array(o) or check_2d_matrix(o):
            return np.array(o)
        else:
            return o


def key_to_str(x):
    if isinstance(x, dict):
        return {str(k): key_to_str(v) for k, v in x.items()}
    else:
        return x


def json_dump(data, file_name):
    with open(file_name, "w") as outfile:
        data = key_to_str(data)
        json.dump(data, outfile, cls=NPEncoder, indent=4)


def to_int(s):
    try:
        return int(s)
    except ValueError:
        return None


def _data_framing(d_in):
    d = d_in.copy()
    ln = -1
    for k, v in d.items():
        b = type(k) == str and type(v) == np.ndarray and len(v.shape) == 1
        if b:
            if ln == -1:
                ln = v.size
            else:
                b &= v.size == ln
        if type(v) == dict:
            d[k] = _data_framing(v)

    if b:
        d = pd.DataFrame.from_dict(d)
    return d


def _int_keys(d_in):
    d = d_in.copy()
    b = True
    for k, v in d.items():
        if b and to_int(k) is None:
            b = False

        if type(v) == dict:
            d[k] = _int_keys(v)

    if b:
        d = {to_int(k): v for k, v in d.items()}
    return d


def json_load(file_name):
    with open(file_name) as json_file:
        data = json.load(json_file, object_hook=np_obj_hook)

    # The order is important! If _int_keys does not convert
    # str(int) keys to int, _data_framing will convert
    # the dictionary to pandas.DataFrame, with columns "1", "23"... (for example)
    data = _int_keys(data)
    data = _data_framing(data)

    return data

# Creating deaths_per_region.csv from raw data ("Dataset-decessi-comunali")

Load all "Dataset-decessi-comunali", massage the data and output that on a single file, <b>deaths_per_region.csv</b> (to be run once - it takes like 25 minutes). <br>
<mark>Note that the ZIP file already contains the processed deaths_per_region.csv</mark>.

In [ ]:
%%time
# Loading deaths...
# Data downloaded at: https://www.istat.it/storage/dati_mortalita/decessi-comunali-giornalieri-regioni(excel)-20-09-2022-5.zip
files = [
    f
    for f in os.listdir(
        os.path.join(
            data_path,
            "Dataset-decessi-comunali-giornalieri_regioni(excel)_5-21-ottobre-2021",
        )
    )
    if f.startswith("comuni_")
]

df = None
for k, f in enumerate(files):
    _df = pd.read_excel(
        os.path.join(
            data_path,
            "Dataset-decessi-comunali-giornalieri_regioni(excel)_5-21-ottobre-2021",
            f,
        ),
        engine="openpyxl",
    )

    year_cols = [c for c in _df.columns if c.startswith("T_") and c[2:].isnumeric()]
    years = [int(c[2:]) for c in year_cols]
    _df = pd.melt(
        _df[["NOME_REGIONE", "GE"] + year_cols],
        id_vars=["NOME_REGIONE", "GE"],
        var_name="year",
    )
    _df = _df.loc[_df["value"] != "n.d.", :]
    _df["value"] = _df["value"].astype(int)
    del year_cols, years
    _df["month"] = _df["GE"] // 100
    _df["day"] = _df["GE"] % 100
    _df["year"] = _df["year"].transform(lambda s: 2000 + int(s[2:]))
    del _df["GE"]
    _df = _df[["year", "month", "day", "value", "NOME_REGIONE"]].rename(
        columns={"value": "deaths", "NOME_REGIONE": "region"}
    )
    _df = (
        _df.groupby(["year", "month", "day", "region"])
        .agg({"deaths": "sum"})
        .reset_index()
        .sort_values(["year", "month", "day", "region"])
        .reset_index(drop=True)
    )
    _df = _df.groupby(["region", "year", "month"]).agg({"deaths": "sum"}).reset_index()

    if df is None:
        df = _df
    else:
        df = pd.concat((df, _df), axis=0, ignore_index=True)

    print(f"File {k + 1} of {len(files)} loaded...")

del files, f, _df

df.to_csv(os.path.join(data_path, "deaths_per_region.csv"), index=False)
del df

# Creating "deaths" DataFrame

Loads <b>deaths</b> DataFrame; for each regione, for each month, for each year in the recording, it gives the recorded number of deaths.

In [ ]:
# This directly loads deaths_per_region.csv
deaths = pd.read_csv(os.path.join(data_path, "deaths_per_region.csv"))
deaths = deaths.rename(columns={"region": "regione"})
deaths.loc[deaths["regione"] == "Emilia-Romagna", "regione"] = "Emilia Romagna"
deaths.loc[
    deaths["regione"] == "Friuli-Venezia Giulia", "regione"
] = "Friuli Venezia Giulia"
deaths.loc[
    deaths["regione"] == "Trentino-Alto Adige/Südtirol", "regione"
] = "Trentino Alto Adige"
deaths.loc[
    deaths["regione"] == "Valle d'Aosta/Vallée d'Aoste", "regione"
] = "Valle d'Aosta"

deaths.head()

# Creating "capoluoghi" DataFrame

Returns <b>capoluoghi</b> (main administrative centres) DataFrame. For each "capoluogo", capoluoghi lists the corresponding regione (region), its geographical coordinates, up to three closest weather stations, and the distance to the weather station.

In [ ]:
# Loads: capoluoghi (with closest weather stations), province, pop_regione, and weather_data
# Data from: https://github.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel
df = pd.read_excel(
    os.path.join(data_path, "italy_geo.xlsx"), engine="openpyxl", sheet_name=1
)
df = df.rename(columns={"lng": "lon", "lat": "lat"})
df["comune"] = df["comune"].astype(str)
df["coords"] = [(x, y) for x, y in zip(df["lat"], df["lon"])]
df = df[["comune", "coords"]]

comuni = df.copy()
del df

comuni.loc[comuni["comune"] == "Massa", "comune"] = "Massa Carrara"
comuni.loc[comuni["comune"] == "Pesaro", "comune"] = "Pesaro Urbino"
comuni.loc[comuni["comune"] == "Iglesias", "comune"] = "Carbonia Iglesias"
comuni.loc[comuni["comune"] == "Monza", "comune"] = "Monza Brianza"
comuni.loc[comuni["comune"] == "Forlì", "comune"] = "Forlì Cesena"

# Data from: https://www.dimmicomefare.it/province-italia-sigle/
df = pd.read_html(
    os.path.join(data_path, "Province.html"),
    keep_default_na=False,
    na_values=["##123%%%"],
)
df = df[0]
df = df.iloc[0:, :]
df.columns = df.iloc[0]
df = df.drop(df.index[0]).reset_index(drop=True)
df = df.rename(
    columns={"Sigla": "sigla", "Provincia": "provincia", "Regione": "regione"}
)

df["provincia"] = df["provincia"].astype(str)
df.loc[df["regione"] == "Valle d’Aosta", "regione"] = "Valle d'Aosta"
df.loc[df["provincia"] == "Valle d'Aosta", "provincia"] = "Aosta"
df["sigla"] = df["sigla"].astype(str).transform(lambda s: s.strip().upper())
df = df[["sigla", "provincia", "regione"]]
df["regione"] = df["regione"].transform(lambda s: s.replace("-", " "))
df["provincia"] = df["provincia"].transform(lambda s: s.replace("-", " "))
province = df.copy()
del df

province = province[
    ~province["provincia"].isin(["Medio Campidano", "Ogliastra"])
].reset_index(drop=True)
province.loc[province["provincia"] == "Barletta Andria Trani", "provincia"] = "Barletta"
province.loc[province["provincia"] == "Olbia Tempio", "provincia"] = "Olbia"
province.loc[province["provincia"] == "Forli-Cesena", "provincia"] = "Forlì"


province["provincia_orig"] = province["provincia"]


def f(s):
    lst = difflib.get_close_matches(
        s,
        comuni["comune"].values,
        n=1,
    )
    if len(lst) > 0:
        return lst[0]
    else:
        return None


province["provincia"] = province["provincia"].transform(f)
province = province.merge(comuni, left_on="provincia", right_on="comune", how="left")
del f

# Data from: https://www.sport-histoire.fr/it/Geografia/Regioni_Italia.php
df = pd.read_html(
    os.path.join(data_path, "Regioni italiane _ regione, capoluogo, superficie.html"),
    keep_default_na=False,
    na_values=["##123%%%"],
)
df = df[0]
df = df.rename(
    columns={
        "Regione": "regione",
        "Capoluogo": "capoluogo",
        df.columns[2]: "superficie",
    }
)
df["regione"] = df["regione"].transform(lambda s: s.replace("-", " "))
df["superficie"] = df["superficie"].astype(str).str.replace("\xa0", "").astype(int)
capoluoghi = df.copy()
del df

capoluoghi["capoluogo"] = capoluoghi["capoluogo"].transform(
    lambda s: difflib.get_close_matches(s, comuni["comune"].values, n=1)[0]
)
capoluoghi = capoluoghi.merge(
    comuni, left_on="capoluogo", right_on="comune", how="left"
)
del capoluoghi["comune"]
del comuni


df = pd.read_csv(os.path.join(data_path, "3007097.csv"))
df = df.rename(
    columns={
        "LATITUDE": "lat",
        "LONGITUDE": "lon",
        "TAVG": "t_avg",
        "TMAX": "t_max",
        "TMIN": "t_min",
        "STATION": "station",
        "NAME": "name",
        "DATE": "date",
        "ELEVATION": "elevation",
    }
)
df["coords"] = [(x, y) for x, y in zip(df["lat"], df["lon"])]
df["year"] = pd.to_datetime(df["date"]).dt.year
df["month"] = pd.to_datetime(df["date"]).dt.month
df["day"] = pd.to_datetime(df["date"]).dt.day
df = df[
    [
        "station",
        "name",
        "year",
        "month",
        "day",
        "t_avg",
        "t_min",
        "t_max",
        "coords",
        "elevation",
    ]
]
weather_data = df.copy()
del df

df = weather_data.drop_duplicates("station")[["station", "coords"]]
df = df.merge(capoluoghi[["capoluogo", "coords"]], how="cross")


def f(p1s, p2s):
    ds = np.zeros((p1s.size,))
    for k, (p1, p2) in enumerate(zip(p1s, p2s)):
        ds[k] = geopy.distance.geodesic(p1, p2).km
    return ds


df["dist"] = f(df["coords_x"].values, df["coords_y"].values)
del f, df["coords_x"], df["coords_y"]

df = (
    df.sort_values(["capoluogo", "dist"])
    .groupby("capoluogo")
    .head(3)
    .reset_index(drop=True)
)
df["dist_min"] = df.groupby("capoluogo")["dist"].transform("min")

df = df[(df["dist"] <= 100.0) | (df["dist"] == df["dist_min"])]
del df["dist_min"]

df.sort_values("dist", ascending=False)
df.sort_values("capoluogo")

capoluoghi = capoluoghi.merge(df, on="capoluogo", how="left")

###########################################################################
# Compute pop_regione
df = pd.read_excel(
    os.path.join(
        data_path,
        "Codici Comuni italiani_1 gennaio 2011.xls",
    ),
    sheet_name="COMUNI 01_01_2011",
)
df = df.rename(
    columns={
        "Codice Regione": "regione_code",
        "Codice Provincia": "provincia_code",
        "Solo denominazione in italiano": "comune",
        "Comune capoluogo di provincia": "is_capoluogo",
        "Popolazione residente al 31/12/2009": "pop",
    }
)
df = df[["regione_code", "provincia_code", "comune", "is_capoluogo", "pop"]]
df = df.merge(
    df[df["is_capoluogo"] == 1][["provincia_code", "comune"]].rename(
        columns={"comune": "capoluogo"}
    ),
    on="provincia_code",
)

df = df.groupby("capoluogo").agg({"regione_code": "first", "pop": "sum"}).reset_index()


def f(s):
    lst = difflib.get_close_matches(
        s,
        np.unique(df["capoluogo"]),
        n=1,
    )
    if len(lst) > 0:
        return lst[0]
    else:
        return None


capoluoghi["x"] = capoluoghi["capoluogo"].transform(f)
del f

d = {row.x: row.capoluogo for row in capoluoghi.itertuples()}
for k, v in d.items():
    if k != v:
        print([k, v])
del capoluoghi["x"]

df["capoluogo_orig"] = df["capoluogo"]
df["capoluogo"] = df["capoluogo"].map(d)
del d


def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    #     return (first, second)

    return first + second == 2


b = isOneToOne(df[~pd.isna(df["capoluogo"])], "capoluogo", "regione_code")
print(f"Is regione_code and capoluogo one-to-one? {b}")
del b

df["capoluogo"] = df.groupby("regione_code")["capoluogo"].transform("first")

d = {row.capoluogo: row.regione for row in capoluoghi.itertuples()}
df["regione"] = df["capoluogo"].map(d)
del d

b = isOneToOne(df, "regione_code", "regione")
print(f"Is regione_code and regione one-to-one? {b}")
del b

df = df.groupby("regione").agg({"pop": "sum"}).reset_index()

pop_regione = df.copy()
del df

# Load Regione-surface
# Data from: https://www.sport-histoire.fr/it/Geografia/Regioni_Italia.php
df = pd.read_html(
    os.path.join(data_path, "Regioni italiane _ regione, capoluogo, superficie.html"),
    keep_default_na=False,
    na_values=["##123%%%"],
)
df = df[0]
df = df.rename(
    columns={
        "Regione": "regione",
        "Capoluogo": "capoluogo",
        df.columns[2]: "surface",
    }
)
df["regione"] = df["regione"].transform(lambda s: s.replace("-", " "))
df["surface"] = df["surface"].astype(str).str.replace("\xa0", "").astype(int)
df = pd.merge(df[["regione", "surface"]], pop_regione, on="regione", how="right")
df["density"] = df["pop"] / df["surface"]

pop_regione = df.copy()
del df
##############################################################################

capoluoghi.head()

# Creating "connectivity" DataFrame

This loads <b>connectivity</b> DataFrame (daily people moving from one regione to another)
r0 = starting region; r1 = destination region; n = number of people making the trip daily; pop0, pop1 = population of r0 and r1 respectively; pil0 and pil1 are the GDPs of r0 and r1; dist is the distance (in km) between the two regions; adjacent = 1 if the two regions do share a border, and 0 otherwise.<br>

In [ ]:
%%time
# Data from: https://www.istat.it/it/archivio/157423
df = pd.read_csv(
    os.path.join(data_path, "matrix_pendo2011_10112014.txt"),
    delimiter=r"\s+",
    names=[
        "tipo_record",
        "tipo_residenza",
        "provincia_residenza",
        "comune_residenza",
        "sesso",
        "motivo",
        "luogo",
        "provincia_lavoro",
        "comune_lavoro",
        "stato_estero_lavoro",
        "mezzo",
        "orario_uscita",
        "durata",
        "numero_stimato",
        "numero",
    ],
    low_memory=False,
)

df = df[["provincia_residenza", "provincia_lavoro", "numero_stimato", "numero"]]
df.loc[df["numero"] == "ND", "numero"] = df.loc[df["numero"] == "ND", "numero_stimato"]
del df["numero_stimato"]
df = df.rename(
    columns={
        "provincia_residenza": "pr0",
        "provincia_lavoro": "pr1",
        "numero": "n",
    }
)

df = df.loc[df["n"] != "ND", :]
df["n"] = df["n"].astype(float)

df = df.groupby(["pr0", "pr1"]).agg({"n": "sum"}).reset_index()
df = df[df["pr0"] != df["pr1"]]

connectivity = df.copy()
del df

df = pd.read_excel(
    os.path.join(
        data_path,
        "Codici Comuni italiani_1 gennaio 2011.xls",
    ),
    sheet_name="COMUNI 01_01_2011",
)
df = df[df["Comune capoluogo di provincia"] == 1].reset_index(drop=True)
df = df[["Codice Provincia", "Solo denominazione in italiano"]]

connectivity = connectivity.merge(
    df, left_on="pr0", right_on="Codice Provincia", how="inner"
)
del connectivity["pr0"], connectivity["Codice Provincia"]
connectivity = connectivity.rename(columns={"Solo denominazione in italiano": "pr0"})

connectivity = connectivity.merge(
    df, left_on="pr1", right_on="Codice Provincia", how="inner"
)
del connectivity["pr1"], connectivity["Codice Provincia"]
connectivity = connectivity.rename(columns={"Solo denominazione in italiano": "pr1"})

connectivity = connectivity[["pr0", "pr1", "n"]]

del df


def f(s):
    lst = difflib.get_close_matches(
        s,
        np.unique(
            np.hstack((connectivity["pr0"].unique(), connectivity["pr1"].unique()))
        ),
        n=1,
    )

    if len(lst) == 0:
        return None
    else:
        return lst[0]


province["x"] = province["provincia"].transform(f)
d = {row.x: row.provincia for row in province.itertuples()}
del province["x"]

connectivity["pr0"] = connectivity["pr0"].map(d)
connectivity["pr1"] = connectivity["pr1"].map(d)
del d

connectivity = connectivity.merge(
    province[["provincia", "regione"]], left_on="pr0", right_on="provincia"
)
del connectivity["provincia"]
connectivity = connectivity.rename(columns={"regione": "r0"})

connectivity = connectivity.merge(
    province[["provincia", "regione"]], left_on="pr1", right_on="provincia"
)
del connectivity["provincia"]
connectivity = connectivity.rename(columns={"regione": "r1"})

connectivity = connectivity[connectivity["r0"] != connectivity["r1"]]

connectivity = connectivity.groupby(["r0", "r1"]).agg({"n": "sum"}).reset_index()

regioni = np.unique(capoluoghi["regione"])
d = {(r.r0, r.r1): r.n for r in connectivity.itertuples()}
for r0 in regioni:
    for r1 in regioni:
        if r1 != r0:
            if not (r0, r1) in d:
                d[(r0, r1)] = 0.0
del r0, r1, regioni
d = [(k[0], k[1], v) for k, v in d.items()]

df = pd.DataFrame(d, columns=["r0", "r1", "n"])
del d

connectivity = df.copy()
del df


# Add population
df = connectivity.copy()
df = df.merge(
    pop_regione[["regione", "pop"]], left_on="r0", right_on="regione", how="left"
)
df = df.rename(columns={"pop": "pop0"})
del df["regione"]

df = df.merge(
    pop_regione[["regione", "pop"]], left_on="r1", right_on="regione", how="left"
)
df = df.rename(columns={"pop": "pop1"})
del df["regione"]

connectivity = df.copy()
del df

# Data source: https://it.wikipedia.org/wiki/Regioni_d%27Italia (retrieved on 10/10/2022)
df = pd.read_html(
    os.path.join(data_path, "Regioni_d'Italia.htm"),
    keep_default_na=False,
    na_values=["##123%%%"],
)
df = df[4]
df = df.rename(
    columns={"Regione o macroregione": "regione", "PIL totale (mln€)": "pil"}
)
df = df[df["PIL Pro capite (macroregione = 100)"] != "-"]
df = df[["regione", "pil"]]
df = df[df["regione"] != "Italia"]
df["regione"] = df["regione"].transform(lambda s: s.replace(" ", " "))
df["regione"] = df["regione"].transform(lambda s: s.replace("-", " "))
df["pil"] = df["pil"].transform(lambda s: s.replace("\xa0", ""))
df["pil"] = df["pil"].astype(float)
df.loc[df["regione"] == "Trentino", "pil"] += df.loc[
    df["regione"] == "Alto Adige", "pil"
].values
df.loc[df["regione"] == "Trentino", "regione"] = "Trentino Alto Adige"
df = df[df["regione"] != "Alto Adige"]
df = df.reset_index(drop=True)

# Check regione name alignment
regioni = {r: 0 for r in np.unique(capoluoghi["regione"])}
for r in df.itertuples():
    if r.regione in regioni:
        regioni[r.regione] += 1
del r
regioni = [(k, v) for k, v in regioni.items()]
if min(regioni, key=lambda c: c[1])[1] == 0:
    print("pil_regione.regione misalignment with capoluoghi!")
del regioni

pil_regione = df.copy()
del df

d = {r.regione: r.pil for r in pil_regione.itertuples()}
df = connectivity.copy()
df["pil0"] = 0.0
df["pil1"] = 0.0
for r in df.itertuples():
    df.loc[r.Index, "pil0"] = d[r.r0]
    df.loc[r.Index, "pil1"] = d[r.r1]

df1 = capoluoghi[["regione", "coords"]].drop_duplicates("regione")
df = (
    df.merge(df1, left_on="r0", right_on="regione", how="left")
    .drop("regione", axis="columns")
    .rename(columns={"coords": "coords0"})
)
df = (
    df.merge(df1, left_on="r1", right_on="regione", how="left")
    .drop("regione", axis="columns")
    .rename(columns={"coords": "coords1"})
)
del df1


def f(p1s, p2s):
    ds = np.zeros((p1s.size,))
    for k, (p1, p2) in enumerate(zip(p1s, p2s)):
        ds[k] = geopy.distance.geodesic(p1, p2).km
    return ds


df["dist"] = f(df["coords0"].values, df["coords1"].values)
del f, df["coords0"], df["coords1"]
connectivity = df.copy()
del df

###################################################################################
# Adjacency between regions
# Data downloaded from: https://www.istat.it/it/files//2015/04/MatriciContiguita_Province_2011_2018.zip
df = pd.read_excel(
    os.path.join(
        data_path,
        "Contiguità_Province2011.xls",
    ),
    sheet_name="Contiguita_Province2011",
)
df
df = df.rename(columns={"PROVINCIA": "pr1", "PROVINCIA ADIACENTE": "pr2"})
df = df[["pr1", "pr2"]]
df = df[(~pd.isna(df["pr1"])) & (~pd.isna(df["pr2"]))].reset_index(drop=True)
df["pr1"] = df["pr1"].astype(str)
df["pr2"] = df["pr2"].astype(str)


def f(s):
    lst = difflib.get_close_matches(
        s,
        province["provincia"],
        n=1,
    )

    if len(lst) == 0:
        return None
    else:
        return lst[0]


d = {s: f(s) for s in np.unique(np.hstack((df["pr1"].unique(), df["pr2"].unique())))}
del f

df["pr1"] = df["pr1"].map(d)
df["pr2"] = df["pr2"].map(d)
del d

df = df.merge(province[["provincia", "regione"]], left_on="pr1", right_on="provincia")
del df["provincia"]
df = df.rename(columns={"regione": "r1"})

df = df.merge(province[["provincia", "regione"]], left_on="pr2", right_on="provincia")
del df["provincia"]
df = df.rename(columns={"regione": "r2"})

df = df[df["r1"] != df["r2"]]
df = df.drop_duplicates(["r1", "r2"])
df = df[["r1", "r2"]].sort_values(["r1", "r2"])

regioni_adjacency = df.copy()
del df
#######################################################################################

df = connectivity.copy()
regioni_adjacency["adjacent"] = 1
df = df.merge(
    regioni_adjacency.rename(columns={"r1": "r0", "r2": "r1"}),
    on=["r0", "r1"],
    how="outer",
)
del regioni_adjacency["adjacent"]
df["adjacent"] = df["adjacent"].fillna(0).astype(int)

connectivity = df.copy()
del df


connectivity.head()

# Creating "deaths_t" DataFrame

It returns <b>deaths_t</b> DataFrame; it gives the same informations (same columns) as <b>deaths</b> DataFrame, adding t_avg, the average temperature in the region in the specific month.

In [ ]:
# It combines capoluoghi and weather_data
df = weather_data.copy()
df = (
    df.groupby(["station", "year", "month"])
    .agg({"t_avg": "mean", "t_min": "min", "t_max": "max"})
    .reset_index()
)

df = df.merge(capoluoghi[["regione", "station", "dist"]], on="station", how="right")
df = (
    df.groupby(["regione", "station"])
    .agg({"t_avg": "mean", "dist": "first"})
    .reset_index()
)


ref_t = df.copy()
del df


df = weather_data.copy()
df = (
    df.groupby(["station", "year", "month"])
    .agg({"t_avg": "mean", "t_min": "min", "t_max": "max"})
    .reset_index()
)

df = df.merge(capoluoghi[["regione", "station", "dist"]], on="station", how="right")


def f(g):
    d = {}

    ws = 1.0 / g["dist"]
    ws /= ws.sum()
    d["t_avg"] = (ws * g["t_avg"].values).sum()

    return pd.Series(d, dtype=object)


df = df.groupby(["regione", "year", "month"]).apply(f).reset_index()
del f

df = df.merge(deaths, on=["regione", "year", "month"], how="inner")


def f(r):
    return datetime(r.year, r.month, 15)


df["date"] = df.apply(f, axis=1)

deaths_t = df.copy()
del df


deaths_t.head()

# Figure 1a

In [ ]:
regioni = ["Lombardia", "Lazio", "Sicilia"]
cmap = {"Lombardia": "#6F69AC", "Lazio": "#FD6F96", "Sicilia": "#95DAC1"}


fig, ax = plt.subplots(figsize=[14, 12])
date_min, date_max = datetime.strptime("31/12/2500", "%d/%m/%Y"), datetime.strptime(
    "01/01/1900", "%d/%m/%Y"
)
for k, regione in enumerate(regioni):
    df = deaths_t[deaths_t["regione"] == regione].copy()
    df = df[df["year"] <= 2019]
    df["day"] = 15
    df["date"] = pd.to_datetime(df[["year", "month", "day"]])

    date_min = min(df["date"].min(), date_min)
    date_max = max(df["date"].max(), date_max)
    ax.plot(
        df["date"],
        df["deaths"] / df["deaths"].mean(),
        "o-",
        #         c=cmap(k),
        c=cmap[regione],
        linewidth=6.0,
        markersize=12.0,
        label=regione,
    )

ax.legend(fontsize=24)
ax.set_ylabel("Norm. death rate", fontsize=24)
ax.tick_params(axis="both", which="both", labelsize=20)
ax.tick_params(axis="x", rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b/%y"))
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))
date_min -= dateutil.relativedelta.relativedelta(months=1)
date_max += dateutil.relativedelta.relativedelta(months=1)
ax.set_xlim(date_min, date_max)

del regioni, cmap, fig, ax, k, regione, df, date_min, date_max

# Figure 2a (it needs JAX)

Add column <i>deaths_pred</i> (number of deaths predicted by the temperature-bi-phasic model) to <b>deaths_t</b> DataFrame.

In [ ]:
t_to_use = "t_avg"

# Bi-phasic model
@jax.jit
def lam_f(params, ts):
    ac, cc, ah, ch, r0 = params
    ac = jnp.exp(ac)
    ah = jnp.exp(ah)
    r0 = jnp.exp(r0)

    ec = jnp.exp(jnp.clip(-ac * ts + cc, a_max=100.0))
    eh = jnp.exp(jnp.clip(ah * ts + ch, a_max=100.0))

    lams = ec + eh  # + r0

    return lams


@jax.jit
def poiss_ll(params, ks, ts):
    ac, cc, ah, ch, r0 = params
    ac = jnp.exp(ac)
    ah = jnp.exp(ah)
    r0 = jnp.exp(r0)

    ec = jnp.exp(jnp.clip(-ac * ts + cc, a_max=100.0))
    eh = jnp.exp(jnp.clip(ah * ts + ch, a_max=100.0))

    lams = ec + eh  # + r0

    ll = (lams - ks * jnp.log(lams)).sum()

    return ll


regioni = list(deaths_t["regione"].unique())
regioni_to_plot = ["Lombardia", "Lazio", "Sicilia"]

for i, regione in enumerate(regioni_to_plot):
    regioni.insert(i, regioni.pop(regioni.index(regione)))
del i, regione

cmap = {"Lombardia": "#6F69AC", "Lazio": "#FD6F96", "Sicilia": "#95DAC1"}


fig, ax = plt.subplots(figsize=[14, 12])

deaths_t["deaths_pred"] = 0.0

for k, regione in enumerate(regioni):
    df = deaths_t[deaths_t["regione"] == regione].copy()
    df = df[df["year"] <= 2019]

    g = lambda params: (
        poiss_ll(
            params,
            ks=df["deaths"].values,
            ts=df[t_to_use].values,
        ),
        jax.grad(
            lambda params: poiss_ll(
                params,
                ks=df["deaths"].values,
                ts=df[t_to_use].values,
            )
        )(params),
    )

    res = minimize(
        g,
        [
            np.log(0.01),
            np.log(0.4 * df["deaths"].mean()),
            np.log(0.1),
            np.log(0.4 * df["deaths"].mean()),
            np.log(0.2 * df["deaths"].mean()),
        ],
        method="TNC",
        jac=True,
        options={"xtol": 10**-6, "maxfun": 10**3},
        callback=None,
    )

    if regione in regioni_to_plot:
        ac, cc, ah, ch, r0 = res.x
        ac = np.exp(ac)
        ah = np.exp(ah)
        r0 = np.exp(r0)

        t0 = (cc - ch + np.log(ac / ah)) / (ac + ah)
        del ac, cc, ah, ch, r0

        lab = f"{regione}"
        ax.plot(
            (df[t_to_use]),
            df["deaths"] / df["deaths"].mean(),
            "o",
            c=cmap[regione],
            label=lab,
            markersize=10,
        )
        del t0, lab

        x = np.linspace(df[t_to_use].min(), df[t_to_use].max(), 1000)
        ax.plot(
            x,
            (lam_f(res.x, x)) / df["deaths"].mean(),
            "-",
            c=cmap[regione],
            linewidth=6.0,
        )
        del x

    deaths_t.loc[deaths_t["regione"] == regione, "deaths_pred"] = lam_f(
        res.x, deaths_t.loc[deaths_t["regione"] == regione, t_to_use].values
    )
del g, df, k, regione, t_to_use

ax.legend(fontsize=20)
ax.set_xlabel("T (C)", fontsize=24)
ax.set_ylabel("Norm. death rate", fontsize=24)
ax.tick_params(axis="both", which="both", labelsize=20)


deaths_t.head()

# Computing corr matrices

Create <b>death_corr</b> DataFrame (with correlation for each pair of regions, both for raw deaths rates and for death reates with the bi-phasic component subtracted - <b>deaths_t</b>["deaths_pred"])

In [ ]:
regioni = sorted(deaths_t["regione"].unique())
df = pd.DataFrame(columns=["r0", "r1", "corr0", "corr"])
i = 0
for k1, regione1 in enumerate(regioni):
    df1 = deaths_t[deaths_t["regione"] == regione1].copy()
    df1 = df1[df1["year"] <= 2019]
    df1 = df1.sort_values(["regione", "year", "month"])
    for k2, regione2 in enumerate(regioni):
        if k2 <= k1:
            pass
        else:
            df2 = deaths_t[deaths_t["regione"] == regione2].copy()
            df2 = df2[df2["year"] <= 2019]
            df2 = df2.sort_values(["regione", "year", "month"])

            c0 = np.corrcoef(
                df1["deaths"],
                df2["deaths"],
            )[0, 1]

            c = np.corrcoef(
                df1["deaths"] - df1["deaths_pred"],
                df2["deaths"] - df2["deaths_pred"],
            )[0, 1]

            i += 1
            df.loc[i] = [regione1, regione2, c0, c]
            i += 1
            df.loc[i] = [regione2, regione1, c0, c]

death_corr = df.copy()
del regioni, df, i, k1, regione1, df1, k2, regione2, df2, c0, c

df = death_corr.copy()
df = df.merge(pop_regione[["regione", "pop"]], left_on="r0", right_on="regione")
del df["regione"]
df = df.rename(columns={"pop": "pop0"})
df = df.merge(pop_regione[["regione", "pop"]], left_on="r1", right_on="regione")
del df["regione"]
df = df.rename(columns={"pop": "pop1"})

death_corr = df.copy()
del df

death_corr.head()

# Figures 1b and 2b

In [ ]:
regioni = sorted(deaths_t["regione"].unique())
regioni = sorted(
    [(r.regione, r.pil) for r in pil_regione.itertuples()], key=lambda c: c[1]
)[::-1]
regioni = [c[0] for c in regioni]

df = death_corr.copy()
d = {r: k for k, r in enumerate(regioni)}
df["i"] = df["r0"].map(d)
df["j"] = df["r1"].map(d)
del d
df = df.sort_values(["i", "j"])
corr0 = np.zeros((len(regioni), len(regioni)))
corr = np.zeros((len(regioni), len(regioni)))
for r in df.itertuples():
    corr0[r.i, r.j] = r.corr0
    corr[r.i, r.j] = r.corr
del r, df

mask = np.zeros_like(corr0, dtype=bool)
np.fill_diagonal(mask, True)
m, M = min(corr0[corr0 > 0.0].min(), corr[corr > 0.0].min()), max(
    corr.max(), corr.max()
)


fig, ax = plt.subplots()
sns.heatmap(
    corr0, vmin=m, vmax=M, xticklabels=regioni, yticklabels=regioni, mask=mask, ax=ax
)


fig, ax = plt.subplots()
ax = sns.heatmap(
    corr, vmin=m, vmax=M, xticklabels=regioni, yticklabels=regioni, mask=mask, ax=ax
)

del mask, fig, ax, corr0, corr, m, M

# Figure 3a (it needs Jax)

In [ ]:
# Fit on n >= 1
df = connectivity.copy()
df = df[(df["dist"] < 50000.0) & (df["n"] >= 1)]


@partial(jax.jit, static_argnames=["b_lognorm"])
def f_jit(params, ns, dists, b_lognorm=False):
    a, b, c = jnp.exp(params)

    lams = a * (jnp.exp(-b * dists) + c)

    if b_lognorm:
        err = ((jnp.log(lams) - jnp.log(ns)) ** 2).sum()
    else:
        err = (lams - ns * jnp.log(lams)).sum()

    return err


params0 = np.array([np.log(4 * 10**4), np.log(1 / 85.0), np.log(10**-5)])

g = lambda params: (
    f_jit(
        params,
        ns=df["n"].values,
        dists=df["dist"].values,
        b_lognorm=True,
    ),
    jax.grad(
        lambda params: f_jit(
            params,
            ns=df["n"].values,
            dists=df["dist"].values,
            b_lognorm=True,
        )
    )(params),
)

res_lognorm = minimize(
    g,
    params0,
    method="TNC",
    jac=True,
    options={"xtol": 10**-8, "maxfun": 10**3},
    callback=None,
)
del g, f_jit, params0

# Show everything
df = connectivity.copy()
df.loc[df["n"] == 0, "n"] = 10**-1

fig, ax = plt.subplots(figsize=[15, 12])

ax.plot(df["dist"], df["n"], "o", c="#557153", markersize=10)

m, M = df["dist"].min(), df["dist"].max()
x = np.linspace(m, M, 10**3)


y = np.exp(res_lognorm.x[0]) * (
    np.exp(-np.exp(res_lognorm.x[1]) * x) + np.exp(res_lognorm.x[2])
)
lab = f"$\propto \mathrm{{e}}^{{-\mathrm{{Dist}} / {np.exp(-res_lognorm.x[1]):1.2f}}} + {np.exp(res_lognorm.x[2]):1.2e}$"
ax.plot(x, y, "-", c="#E6E5A3", linewidth=6.0, label=lab)
del m, M, x, y, lab

ax.set_ylim([df["n"].min() * 0.5, None])
ax.set_yscale("log")
ax.legend(fontsize=24)
ax.set_xlabel("Distance (km)", fontsize=24)
ax.set_ylabel(
    r"$c_{ij}$ (# commuters, region $j$ $\rightarrow$ region $i{}$)", fontsize=24
)
ax.tick_params(axis="both", which="both", labelsize=20)

# Figure 3b (it needs Jax)

In [ ]:
df = connectivity.copy()

df = df.groupby("r1").agg({"n": "sum", "pil1": "first", "pop1": "first"}).reset_index()
df["pil1"] /= 10**3


def f(params, ns, pil):
    a = params

    lams = a * pil
    err = ((jnp.log(lams) - jnp.log(ns)) ** 2).sum()
    return err


f_jit = jax.jit(f)

params0 = np.array([200.0])
g = lambda params: (
    f_jit(
        params,
        ns=df["n"].values,
        pil=df["pil1"].values,
    ),
    jax.grad(
        lambda params: f_jit(
            params,
            ns=df["n"].values,
            pil=df["pil1"].values,
        )
    )(params),
)

res = minimize(
    g,
    params0,
    method="TNC",
    jac=True,
    options={"xtol": 10**-6, "maxfun": 10**3},
    callback=None,
)
del f, g, f_jit, params0

if not res.success:
    print(res)

fig, ax = plt.subplots(figsize=[15, 12])

ax.plot(df["pil1"], df["n"] / 10**5, "o", c="#557153", markersize=10)

m, M = df["pil1"].min(), df["pil1"].max()
x = np.linspace(m, M, 10**3)
y = res.x[0] * x

lab = f"{signif_digit(res.x[0], 2):1.0f} * $\mathrm{{GDP}}_{{i}}$"
ax.plot(x, y / 10**5, "-", c="#E6E5A3", linewidth=6.0, label=lab)
del m, M, x, y, lab

ax.set_xlim([0.0, None])
ax.set_ylim([0.0, None])
ax.legend(fontsize=24)
ax.set_xlabel(r"$\mathrm{GDP}_{i}$ (M€)", fontsize=24)
ax.set_ylabel(r"$c_{i:}$ (# commuters to region i) ($x 10^5$)", fontsize=24)
ax.tick_params(axis="both", which="both", labelsize=20)

# Figure 4 (it needs Jax)

In [ ]:
df = connectivity.copy()
df = df[(df["dist"] < 50000.0) & (df["n"] >= 1)]


@jax.jit
def f_jit(params, ns, dists, pop0s, pil1s):
    a, b = jnp.exp(params)

    lams = a * pil1s * pop0s * jnp.exp(-b * dists)

    err = ((jnp.log(lams) - jnp.log(ns)) ** 2).sum()

    return err


params0 = np.array([np.log(0.01), np.log(1 / 85.0)])
g = lambda params: (
    f_jit(
        params,
        ns=df["n"].values,
        dists=df["dist"].values,
        pop0s=df["pop0"].values,
        pil1s=df["pil1"].values,
    ),
    jax.grad(
        lambda params: f_jit(
            params,
            ns=df["n"].values,
            dists=df["dist"].values,
            pop0s=df["pop0"].values,
            pil1s=df["pil1"].values,
        )
    )(params),
)

res = minimize(
    g,
    params0,
    method="TNC",
    jac=True,
    options={"xtol": 10**-8, "maxfun": 10**3},
    callback=None,
)

if not res.success:
    print(res)

a, b = jnp.exp(res.x)
print(f"a = {a:1.3e}, tau = {1 / b:1.3f}")
lams = a * df["pil1"].values * df["pop0"].values * np.exp(-b * df["dist"].values)

fig, ax = plt.subplots(figsize=[12, 12])
ax.axis("equal")

ax.plot(df["n"], lams, "o", c="#557153", markersize=10)
ax.set_xscale("log")
ax.set_yscale("log")

mx, Mx = ax.get_xlim()
my, My = ax.get_ylim()

m, M = min(df["n"].min(), lams.min()), max(df["n"].max(), lams.max())

ax.plot([m, M], [m, M], "-", c="#E6E5A3", linewidth=6.0, label="y=x")
ax.set_xlim(mx, Mx)
ax.set_ylim(my, My)
del m, mx, my, M, Mx, My

ax.legend(fontsize=24)
ax.set_xlabel(r"$c_{ij}$", fontsize=24)
ax.set_ylabel(
    f"$c_{{ij}}^{{\mathrm{{fit}}}} \propto \mathrm{{pop}}_{{j}} \, "
    + f"\mathrm{{GDP}}_{{i}} \, \mathrm{{e}}^{{-\mathrm{{dist}}_{{ij}} / {1 / b:1.1f}}}$",
    fontsize=24,
)
ax.tick_params(axis="both", which="both", labelsize=20)
del a, b

del fig, ax

# SIR (full model) functions (it needs Jax)

In [ ]:
# (jitted) Functions
@jax.jit
def sig(x):
    return 1.0 / (1.0 + jnp.exp(-x))


@jax.jit
def inv_sig(y):
    return jnp.log(y / (1.0 - y))


@jax.jit
def lambda_update_step(s_i_r, betas, gamma, pops, c, c_sum):
    S0, I0, R0 = s_i_r

    M = jnp.diag(S0 * (1 - c_sum)) + c * (S0 / pops)
    M_sum = M.sum(axis=1)
    dI = betas / pops * M_sum * (I0 * (1.0 - c_sum) + (c @ (I0 / pops)))
    M = M.T / M_sum
    dI = M @ dI

    S = S0 - dI
    I = I0 + dI - gamma * I0

    R = R0 + gamma * I0
    return (S, I, R), (S, I, R)


@jax.jit
def beta_t_f(abeta, bbeta, ts, densities):
    betas = jnp.clip(
        jnp.log(densities) * jnp.exp(-abeta * ts + bbeta), a_max=1.0 - 10**-4
    )
    return betas


@partial(jax.jit, static_argnames=["n_steps_per_month", "r_nb"])
def lambda_update_month(
    err,
    data,
    n_steps_per_month,
    abeta,
    bbeta,
    gamma,
    mu,
    ah,
    bh,
    rate0s,
    pops,
    densities,
    c,
    c_sum,
    r_nb=None,
):
    ns, ts, S0, I0, R0, w = data
    betas = beta_t_f(abeta, bbeta, ts, densities)
    (S, I, R), s_i_rs = jax.lax.scan(
        lambda s_i_r, dummy: lambda_update_step(s_i_r, betas, gamma, pops, c, c_sum),
        (S0, I0, R0),
        None,
        length=n_steps_per_month,
    )

    deaths_spreading = mu * R
    deaths_hot = pops * jnp.exp(ah * ts + bh)
    deaths_base = pops * rate0s

    mean_deaths = mu * R + deaths_hot + deaths_base
    if r_nb is None:  # Poisson
        _err = w * (mean_deaths - ns - ns * jnp.log(mean_deaths / ns)).sum()
    elif r_nb > 0.0:
        ls = mean_deaths
        _err = (
            w
            * (
                -ns * (jnp.log(ls / (ls + r_nb)) - jnp.log(ns / (ns + r_nb)))
                - r_nb * (jnp.log(r_nb / (ls + r_nb)) - jnp.log(r_nb / (ns + r_nb)))
            ).sum()
        )
    else:  # Gaussian
        _err = (-r_nb) * w * ((mean_deaths - ns) ** 2).sum()

    err += _err
    return err, (
        S,
        I,
        R,
        mean_deaths,
        _err,
        s_i_rs,
        deaths_spreading,
        deaths_hot,
        deaths_base,
    )


@jax.jit
def flat_prior(x, params):
    x0, x1, alpha = params

    def f0(x):
        return alpha * (x - x0) ** 2

    def f1(x):
        return alpha * (x - x1) ** 2

    def scan_f(err, x):
        _err = jax.lax.cond(
            x < x0,
            f0,
            lambda x: jax.lax.cond(
                x > x1,
                f1,
                lambda x: 0.0,
                x,
            ),
            x,
        )

        err += _err
        return err, _err

    err, _ = jax.lax.scan(scan_f, 0.0, x)
    err /= x.size

    return err


@partial(jax.jit, static_argnames=["n_steps_per_month", "r_nb"])
def sir_vec(
    params,
    n_steps_per_month,
    ns,
    ts,
    S0,
    I0,
    R0,
    c0,
    pil1_pop0_m,
    dist_m,
    pops,
    densities,
    ws=None,
    lambda_corr=None,
    corr_array=None,
    corr_idxs=None,
    r_nb=None,
):
    alpha, tau, abeta, bbeta, gamma, mu, ah, bh = params[:8]
    rate0s = params[8 : 8 + ns.shape[1]]

    alpha = jnp.exp(alpha)
    tau = jnp.exp(tau)
    abeta = jnp.exp(abeta)
    gamma = jnp.exp(gamma)
    mu = sig(mu)

    ah = jnp.exp(ah)
    rate0s = jnp.exp(rate0s)

    c = c0 + alpha * pil1_pop0_m * jnp.exp(-dist_m / tau)

    c_sum = c.sum(axis=0) / pops

    if ws is None:
        ws = np.ones((ns.shape[0],))

    scan_f = lambda s_i_r_err, data: lambda_update_month(
        s_i_r_err,
        data,
        n_steps_per_month,
        abeta,
        bbeta,
        gamma,
        mu,
        ah,
        bh,
        rate0s,
        pops,
        densities,
        c,
        c_sum,
        r_nb,
    )
    _, s_i_r_ds_errs = jax.lax.scan(
        scan_f,
        0.0,
        (
            ns,
            ts,
            S0,
            I0,
            R0,
            ws,
        ),
    )

    errs = s_i_r_ds_errs[4]

    if lambda_corr is not None:
        deaths = s_i_r_ds_errs[3]

        if r_nb is None:
            irnb = 0.0
        else:
            irnb = 1 / r_nb  # It does not work for r_nb < 0!!

        def scan_f(errc, cij):
            corr0, (i, j) = cij

            x, y = deaths[:, i], deaths[:, j]

            _corr = ((x - x.mean()) * (y - y.mean())).mean() / (
                jnp.sqrt(x.var() + x.mean() + (x**2).mean() * irnb)
                * jnp.sqrt(y.var() * (1 + irnb) + y.mean())
            )

            errc += 0.5 * (_corr - corr0) ** 2

            return errc, None

        errc, _ = jax.lax.scan(scan_f, 0.0, (corr_array, corr_idxs))
        errc /= corr_array.shape[0]

        errs += lambda_corr * errc / errs.size

    return errs


@partial(jax.jit, static_argnames=["n_steps_per_month", "r_nb"])
def sir(
    params,
    n_steps_per_month,
    ns,
    ts,
    S0,
    I0,
    R0,
    c0,
    pil1_pop0_m,
    dist_m,
    pops,
    densities,
    ws=None,
    lambda_corr=None,
    corr_array=None,
    corr_idxs=None,
    c_sum_prior=None,
    r0_prior=None,
    tau_prior=None,
    kappa_prior=None,
    dt_beta_prior=None,
    mu_prior=None,
    r_nb=None,
):
    alpha, tau, abeta, bbeta, gamma, mu, ah, bh = params[:8]
    rate0s = params[8 : 8 + ns.shape[1]]

    alpha = jnp.exp(alpha)
    tau = jnp.exp(tau)
    abeta = jnp.exp(abeta)
    gamma = jnp.exp(gamma)
    mu = sig(mu)

    ah = jnp.exp(ah)
    rate0s = jnp.exp(rate0s)

    c = c0 + alpha * pil1_pop0_m * jnp.exp(-dist_m / tau)

    c_sum = c.sum(axis=0) / pops

    if ws is None:
        ws = np.ones((ns.shape[0],))

    scan_f = lambda s_i_r_err, data: lambda_update_month(
        s_i_r_err,
        data,
        n_steps_per_month,
        abeta,
        bbeta,
        gamma,
        mu,
        ah,
        bh,
        rate0s,
        pops,
        densities,
        c,
        c_sum,
        r_nb,
    )
    err, s_i_r_ds_errs = jax.lax.scan(
        scan_f,
        0.0,
        (
            ns,
            ts,
            S0,
            I0,
            R0,
            ws,
        ),
    )

    err /= ns.size

    if dt_beta_prior is not None:
        err += dt_beta_prior[1] * (1 / abeta - dt_beta_prior[0]) ** 2

    if c_sum_prior is not None:
        err += flat_prior(c_sum, c_sum_prior)

    if r0_prior is not None:
        r0s = beta_t_f(abeta, bbeta, ts, densities).flatten() / gamma
        err += flat_prior(r0s, r0_prior)

    if tau_prior is not None:
        err += flat_prior(jnp.array([tau]), tau_prior)

    if kappa_prior is not None:
        err += flat_prior(jnp.array([alpha * tau]), kappa_prior)

    if mu_prior is not None:
        err += flat_prior(jnp.array([mu]), mu_prior)

    if lambda_corr is not None:
        deaths = s_i_r_ds_errs[3]

        if r_nb is None:
            irnb = 0.0
        else:
            irnb = 1 / r_nb  # It does not work for r_nb < 0!!

        def scan_f(errc, cij):
            corr0, (i, j) = cij

            x, y = deaths[:, i], deaths[:, j]

            _corr = ((x - x.mean()) * (y - y.mean())).mean() / (
                jnp.sqrt(x.var() + x.mean() + (x**2).mean() * irnb)
                * jnp.sqrt(y.var() * (1 + irnb) + y.mean())
            )

            errc += 0.5 * (_corr - corr0) ** 2

            return errc, None

        errc, _ = jax.lax.scan(scan_f, 0.0, (corr_array, corr_idxs))
        errc /= corr_array.shape[0]

        err += lambda_corr * errc

    return err


@partial(jax.jit, static_argnames=["n_steps_per_month", "r_nb"])
def sir_gen(
    params,
    n_steps_per_month,
    ns,
    ts,
    S0,
    I0,
    R0,
    c0,
    pil1_pop0_m,
    dist_m,
    pops,
    densities,
    r_nb=None,
):
    alpha, tau, abeta, bbeta, gamma, mu, ah, bh = params[:8]
    rate0s = params[8 : 8 + ns.shape[1]]

    alpha = jnp.exp(alpha)
    tau = jnp.exp(tau)
    abeta = jnp.exp(abeta)
    gamma = jnp.exp(gamma)
    mu = sig(mu)

    ah = jnp.exp(ah)
    rate0s = jnp.exp(rate0s)

    c = c0 + alpha * pil1_pop0_m * jnp.exp(-dist_m / tau)

    c_sum = c.sum(axis=0) / pops

    scan_f = lambda s_i_r_err, data: lambda_update_month(
        s_i_r_err,
        data,
        n_steps_per_month,
        abeta,
        bbeta,
        gamma,
        mu,
        ah,
        bh,
        rate0s,
        pops,
        densities,
        c,
        c_sum,
        r_nb,
    )

    err, s_i_r_ds = jax.lax.scan(
        scan_f,
        0.0,
        (ns, ts, S0, I0, R0, jnp.ones((ns.shape[0],))),
    )

    return s_i_r_ds[:3], s_i_r_ds[3], s_i_r_ds[5], s_i_r_ds[6:9]


def reals_to_params(reals):
    alpha, tau, abeta, bbeta, gamma, mu, ah, bh = reals[:8]
    rate0s = reals[8:]

    params = [
        jnp.log(alpha),
        jnp.log(tau),
        jnp.log(abeta),
        bbeta,
        jnp.log(gamma),
        inv_sig(mu),
        jnp.log(ah),
        bh,
    ] + [jnp.log(rate0) for rate0 in rate0s]
    return params


def params_to_reals(params):
    alpha, tau, abeta, bbeta, gamma, mu, ah, bh = params[:8]
    rate0s = params[8:]

    reals = np.array(
        [
            jnp.exp(alpha),
            jnp.exp(tau),
            np.exp(abeta),
            bbeta,
            jnp.exp(gamma),
            sig(mu),
            jnp.exp(ah),
            bh,
        ]
        + [jnp.exp(rate0) for rate0 in rate0s]
    )

    return reals

# Load optimised parameters from file

In [ ]:
# Reloading minimize_data from file
minimize_data = json_load(
    os.path.join(
        data_path,
        "minimize_data.json",
    )
)
var_names = [
    "regioni",
    "deaths",
    "temperatures",
    "c0",
    "pops",
    "densities",
    "pils",
    "dist_m",
    "pil1_pop0_m",
    "n_steps_a_day",
    "clip_values",
    "ps",
    "c_sum_prior",
    "tau_prior",
    "kappa_prior",
    "r0_prior",
    "dt_beta_prior",
    "lambda_corr",
    "delta_corr_init_step",
    "n_train_for_corr",
    "corr_array",
    "corr_idxs",
    "test_idxs",
    "train_idxs",
    "adam_data",
    "params0",
    "minimize_res",
    "minimize_res_output",
    "r_nb",
]
for var in var_names:
    if var not in minimize_data:
        print(var)
    else:
        locals()[var] = minimize_data[var]
del var, var_names

S0s = np.tile(pops, pops.size).reshape(pops.size, pops.size)
I0s = np.zeros_like(S0s)
R0s = np.zeros_like(S0s)

for i in range(pops.size):
    S0s[i, i] -= 1
    I0s[i, i] = 1
del i

# Print parameters summary

In [ ]:
teh = minimize_res["test_error_history"]
te_steps = np.array([int(s) for s in teh.keys()])
te_values = np.array([e for e in teh.values()])
del teh

min_step = te_steps[-1]

print(f"min_step = {min_step}")
print()
params_opt = minimize_res["params_history"][min_step]
del te_steps, te_values, min_step

reals = params_to_reals(params_opt)
kappa, d0, abeta, bbeta, gamma, mu, ah, bh = reals[:8]
ro0s = reals[8:]
del reals


c = c0 + kappa * pil1_pop0_m * np.exp(-dist_m / d0)
c_sum = c.sum(axis=0) / pops

print(f"kappa = {kappa:1.3f}")
print(f"d0 = {d0:1.1f} Km")
print(f"Non-commuters account for {(c - c0).sum() / c.sum() * 100:1.3f}% of all flux")
print(
    f"Commuters fraction: max {c_sum.max()} in {regioni[c_sum.argmax()]}, "
    + f" min {c_sum.min()} in {regioni[c_sum.argmin()]}"
)
print()

print(f"mu = {mu * 10**5:1.3f} deaths/10^5")
print()

print(
    f"gamma = {gamma:1.4f}; " + f"recovery half-life = {np.log(2.0) / gamma:1.1f} days"
)
print()

if bbeta < 0.0:
    print(f"beta = exp(-(T + {-bbeta / abeta:1.1f} °C) / {1 / abeta:1.1f} °C)")
else:
    print(f"beta = exp(-(T - {bbeta / abeta:1.1f} °C) / {1 / abeta:1.1f} °C)")

betas = beta_t_f(abeta, bbeta, temperatures, densities)
print(f"<beta> = {betas.mean()}")

print(
    f"Doubling time of disease at {temperatures.flatten()[betas.argmax()]:1.2f} °C "
    + f"= {np.log(2.0) / betas.max():1.1f} days"
)
print(
    f"Doubling time of disease at {temperatures.flatten()[betas.argmin()]:1.2f} °C "
    + f"= {np.log(2.0) / betas.min():1.1f} days"
)

print(
    f"R0 at {temperatures.flatten()[betas.argmax()]:1.2f} °C: "
    + f"{betas.max() / gamma}"
)
print(
    f"R0 at {temperatures.flatten()[betas.argmin()]:1.2f} °C: "
    + f"{betas.min() / gamma}"
)


del kappa, d0, abeta, bbeta, gamma, mu, ah, bh, ro0s
del c, c_sum, betas

# Plot error during training

In [ ]:
teh = minimize_res["test_error_history"]
te_steps = np.array([int(s) for s in teh.keys()])
te_values = np.array([e for e in teh.values()])
del teh

y = minimize_res["values"]
tre_values = np.zeros_like(te_values)

s0 = 0
for k in range(te_steps.size):
    s1 = te_steps[k]
    tre_values[k] = y[s0:s1].mean()
del s0, s1, k
del y

fig, ax = plt.subplots()
ax1 = ax.twinx()
idxs = te_steps >= 0
ax.plot(te_steps[idxs], te_values[idxs], "bo-")
ax1.plot(te_steps[idxs], tre_values[idxs], "rs-")
ax.set_xscale("log")
ax.set_xlabel("Training step")
ax.set_ylabel("Test error")
ax1.set_ylabel("Train error")

ax.yaxis.label.set_color("blue")
ax1.spines["left"].set_color("blue")
ax.tick_params(axis="y", colors="blue")

ax1.yaxis.label.set_color("red")
ax1.spines["right"].set_edgecolor("red")
ax1.tick_params(axis="y", colors="red")

del te_steps, te_values, tre_values, idxs
del fig, ax, ax1

# Compute model's predictions (for Figures 5 and 6; they all need Jax)

In [ ]:
regioni = list(deaths_t["regione"].unique())

teh = minimize_res["test_error_history"]
te_steps = np.array([int(s) for s in teh.keys()])
te_values = np.array([e for e in teh.values()])
del teh

min_step = te_steps[-1]
params_res = minimize_res["params_history"][min_step]
del te_steps, te_values, min_step

s_i_rs = None
ds = None
death_parts = None
ev = None
for idx in range(len(regioni)):
    _s_i_rs, _ds, _ev, _death_parts = sir_gen(
        params_res,
        30 * n_steps_a_day,
        deaths,
        temperatures,
        S0s[np.repeat(idx, deaths.shape[0]), :],
        I0s[np.repeat(idx, deaths.shape[0]), :],
        R0s[np.repeat(idx, deaths.shape[0]), :],
        c0,
        pil1_pop0_m,
        dist_m,
        pops,
        densities,
    )

    if s_i_rs is None:
        s_i_rs = [_s_i_rs[k] * ps[idx] for k in range(3)]
        ds = _ds * ps[idx]
        death_parts = [_death_parts[k] * ps[idx] for k in range(3)]
        ev = [_ev[k] * ps[idx] for k in range(3)]
    else:
        for k in range(3):
            s_i_rs[k] += _s_i_rs[k] * ps[idx]
            death_parts[k] += _death_parts[k] * ps[idx]
            ev[k] += _ev[k] * ps[idx]
        ds += _ds * ps[idx]
del _s_i_rs, _ds, k, idx

# Figure 5a

In [ ]:
fig, ax = plt.subplots(figsize=[12, 12])
ax.plot(deaths[:], ds[:], "o", c="#FB2576", markersize=10)
m, M = min(deaths.min(), ds.min()), max(deaths.max(), ds.max())
ax.plot([m, M], [m, M], "-", c="#150050", linewidth=6.0, label="y=x")

ax.set_xlim([m, M])
ax.set_ylim([m, M])

del m, M

ax.legend(fontsize=24)
ax.set_xlabel(r"deaths (per month)", fontsize=24)
ax.set_ylabel(r"deaths (predicted)", fontsize=24)
ax.tick_params(axis="both", which="both", labelsize=20)

print(f"death corr: {np.corrcoef(deaths.flatten(), ds.flatten())[0, 1]:1.4f}")

# Figure 5b

In [ ]:
np.random.seed(89873903)

fig, ax = plt.subplots(figsize=[12, 12])

cmap = {"Lombardia": "#6F69AC", "Lazio": "#FD6F96", "Sicilia": "#95DAC1"}

regioni_to_plot = ["Sicilia", "Lazio", "Lombardia"]

df = deaths_t[np.isin(deaths_t["regione"], regioni_to_plot)].copy()
df = df[df["year"] <= 2019]

date_min, date_max = df["date"].min(), df["date"].max()
dates = pd.date_range(date_min, date_max, freq="SM")[::2]
del df

mms = []
preds = []
for regione in regioni_to_plot:
    n = regioni.index(regione)
    if r_nb is None:
        y = np.random.poisson(ds[:, n]) / pops[n]
    elif r_nb > 0.0:
        y = np.random.negative_binomial(r_nb, r_nb / (ds[:, n] + r_nb)) / pops[n]

    preds.append(y)
    m = min(y.min(), deaths[:, n].min() / pops[n])
    M = max(y.max(), deaths[:, n].max() / pops[n])
    mms.append((m, M, M - m))
del m, M

offsets = np.zeros((len(regioni_to_plot),))
for i in range(1, len(regioni_to_plot)):
    offsets[i] = (
        offsets[i - 1] + -mms[i][0] + mms[i - 1][1] - 0.1 * (mms[i - 1][2] + mms[i][2])
    )


for i, regione in enumerate(regioni_to_plot):
    n = regioni.index(regione)
    y = preds[i]
    ax.plot(
        dates,
        deaths[:, n] / pops[n] + offsets[i],
        "o--",
        c=cmap[regione],
        markersize=8,
        linewidth=3.0,
    )
    ax.plot(
        dates,
        y + offsets[i],
        "-",
        c=cmap[regione],
        linewidth=6.0,
        label=f"{regione}",
    )

hs, ls = ax.get_legend_handles_labels()
hs = [hs[2], hs[1], hs[0]]
ls = [ls[2], ls[1], ls[0]]
ax.legend(hs, ls, fontsize=24)
del hs, ls

ax.set_ylabel("deaths (offset)", fontsize=24)
ax.get_yaxis().set_ticks([])
ax.tick_params(axis="both", which="both", labelsize=20)

ax.tick_params(axis="both", which="both", labelsize=20)
ax.tick_params(axis="x", rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b/%y"))
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1, 7]))
date_min -= dateutil.relativedelta.relativedelta(months=1)
date_max += dateutil.relativedelta.relativedelta(months=1)
ax.set_xlim(date_min, date_max)

del regione, n, regioni_to_plot, y, mms, preds, i, offsets
del date_min, date_max

# Figure 6

In [ ]:
df = death_corr.copy()
d = {r: k for k, r in enumerate(regioni)}
df["i"] = df["r0"].map(d)
df["j"] = df["r1"].map(d)
del d

df = df[df["i"] < df["j"]].sort_values(["i", "j"])

corr0 = np.zeros((len(regioni) * (len(regioni) - 1) // 2))
k = 0
for r in df.itertuples():
    if r.i < r.j:
        corr0[k] = r.corr0
        k += 1
del r, df, k

corr = np.zeros_like(corr0)
corr_lambda = np.zeros_like(corr0)
k = 0
if r_nb is None:
    irnb = 0.0
elif r_nb > 0.0:
    irnb = 1 / r_nb
else:
    irnb = None

for i in range(len(regioni)):
    for j in range(i + 1, len(regioni)):
        x, y = np.array(ds[:, i]), np.array(ds[:, j])
        if irnb is not None:
            corr[k] = ((x - x.mean()) * (y - y.mean())).mean() / (
                np.sqrt(x.var() + x.mean() + (x**2).mean() * irnb)
                * np.sqrt(y.var() * (1 + irnb) + y.mean())
            )
        else:
            corr[k] = np.corrcoef(x, y)[0, 1]
        corr_lambda[k] = np.corrcoef(x, y)[0, 1]
        k += 1
del k, i, j, x, y, irnb


fig, ax = plt.subplots(figsize=[12, 12])
ax.plot(corr0, corr, "o", c="#0008C1", markersize=10)
m, M = min(corr0.min(), corr.min()), 1.0
ax.plot([m, M], [m, M], "-", c="#E6CBA8", linewidth=6.0, label="y=x")
ax.set_xlim([m, M])
ax.set_ylim([m, M])
del m, M

ax.legend(fontsize=24)
ax.set_xlabel(r"region-region correlation", fontsize=24)
ax.set_ylabel(r"region-region correlation (predicted)", fontsize=24)
ax.tick_params(axis="both", which="both", labelsize=20)

print(f"corr(corr) = {np.corrcoef(corr0, corr)[0, 1]:1.4f}")
print(f"corr < corr0 = {(corr <= corr0).mean() * 100:1.1f}%")

# Optimisation of the full model

## Pre-computing the data we need for the optimisation

In [ ]:
regioni = sorted(deaths_t["regione"].unique())

df = deaths_t.loc[deaths_t["year"] <= 2019].copy()
df = df.sort_values(["regione", "year", "month"]).reset_index(drop=True)
x = df[["deaths", "t_avg"]].values
deaths = x[:, 0].reshape(len(regioni), x.shape[0] // len(regioni)).T
temperatures = x[:, 1].reshape(len(regioni), x.shape[0] // len(regioni)).T
del x, df


d = {r: k for k, r in enumerate(regioni)}
df = connectivity.copy()
df["j"] = df["r0"].map(d)
df["i"] = df["r1"].map(d)
del d

c0 = np.zeros((len(regioni), len(regioni)))
for r in df.itertuples():
    c0[r.i, r.j] = r.n
del r

pop_dict = {r.regione: (r.pop, r.density) for r in pop_regione.itertuples()}
pops = np.zeros((len(regioni),))
densities = 10.0 * np.ones((len(regioni),))  # 10 is just not to have 1
for i, r in enumerate(regioni):
    pops[i] = pop_dict[r][0]
#     densities[i] = pop_dict[r][1]
del pop_dict, i, r
densities = np.exp(np.log(densities) / np.log(densities).mean())

pil_dict = {r.regione: r.pil for r in pil_regione.itertuples()}
pils = np.zeros((len(regioni),))
for i, r in enumerate(regioni):
    pils[i] = pil_dict[r]
del pil_dict, i, r

df = connectivity.copy()
d = {r: k for k, r in enumerate(regioni)}
df["i"] = df["r0"].map(d)
df["j"] = df["r1"].map(d)
del d
df = df.sort_values(["i", "j"])
dist_m = np.zeros((len(regioni), len(regioni)))
for r in df.itertuples():
    dist_m[r.i, r.j] = r.dist
del r, df

pil1_pop0_m = np.ones((len(regioni), len(regioni)))
pil1_pop0_m = (pils * pil1_pop0_m.T).T * pops
np.fill_diagonal(pil1_pop0_m, 0.0)
pil1_pop0_m *= c0.mean() / (pil1_pop0_m * np.exp(-dist_m / 100.0)).mean()

## Optimisation

Expect around 3 hours for a 10^6-step optimisation.

In [ ]:
# %%time
print_ = str_print()

n_steps_a_day = 1

clip_values = [-100.0, 100.0]

r_nb = None

ps = pils / pils.sum()

S0s = np.tile(pops, pops.size).reshape(pops.size, pops.size)
I0s = np.zeros_like(S0s)
R0s = np.zeros_like(S0s)

for i in range(pops.size):
    S0s[i, i] -= 1
    I0s[i, i] = 1
del i

### Setting priors ###
r0_prior = None
tau_prior = None
kappa_prior = None
dt_beta_prior = None
mu_prior = None
lambda_corr = None

n = np.median(deaths)
delta_n = 10**-2 * n

c_sum_prior = [0.0, 0.2, None]
delta_c_sum = 10**-5
c_sum_prior[2] = delta_n**2 / (2 * n) / delta_c_sum**2
del delta_c_sum

r0_prior_start = r0_prior

tau_prior = [-1.0, 0.0, None]
delta_tau = 300.0
tau_prior[2] = delta_n**2 / (2 * n) / delta_tau**2
del delta_tau

delta_corr_init_step = 1 * 10**4
n_train_for_corr = 100
delta_corr = 0.01
lambda_corr = delta_n**2 / (2 * n) / delta_corr**2
del delta_corr

del n, delta_n

###


# creating corr vector
df = death_corr.copy()
d = {r: k for k, r in enumerate(regioni)}
df["i"] = df["r0"].map(d)
df["j"] = df["r1"].map(d)
del d

df = df[df["i"] < df["j"]].sort_values(["i", "j"])
corr_array = np.zeros((len(regioni) * (len(regioni) - 1) // 2,))
corr_idxs = np.zeros((len(regioni) * (len(regioni) - 1) // 2, 2), dtype=int)
k = 0
for r in df.itertuples():
    if r.i < r.j:
        corr_array[k] = r.corr0
        corr_idxs[k, :] = r.i, r.j
        k += 1
del r, df, k

# As test indexes I want at least one examplar for each month (12 examplars).
# If you have more than 12 years, you just keep one random month for each year.
# If years are (as in our case) less than 12, some of the years will contribute
# more than one month, paying attention - in this case - that the contributed
# months were 6 months apart (like June and December, or April and October).
test_idxs = []
ys = np.arange(deaths.shape[0] // 12)
np.random.shuffle(ys)
ky = 0
for m in range(max(12, ys.size)):
    if ky < ys.size:
        _ky = ky
    else:
        _ky = (ky + 6) % 12

    ky += 1

    _m = m % 12

    test_idxs.append(ys[_ky] * 12 + _m)
del ys, ky, m, _ky, _m

test_idxs = np.array(test_idxs)

months = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]
print_(", ".join([f"{months[k % 12]} {k // 12 + 2011}" for k in test_idxs]))
print_()
del months


def test_data_generator():
    regione_idxs = np.repeat(np.arange(len(regioni)), test_idxs.size)
    idxs = np.tile(test_idxs, len(regioni))

    yield (
        deaths[idxs],
        temperatures[idxs],
        S0s[regione_idxs, :],
        I0s[regione_idxs, :],
        R0s[regione_idxs, :],
    ), {
        "ws": ps[regione_idxs],
        "lambda_corr": lambda_corr,
        "c_sum_prior": None,
        "r0_prior": None,
        "tau_prior": None,
        "kappa_prior": None,
        "dt_beta_prior": None,
        "mu_prior": None,
    }


train_idxs = np.setdiff1d(np.arange(deaths.shape[0]), test_idxs)

adam_data = {"params": None}


def train_data_generator(adam_data=None):
    n_calls = 0

    while True:
        idxs = np.random.choice(
            train_idxs,
            replace=True,
            size=10
            if lambda_corr is None or n_calls < delta_corr_init_step
            else n_train_for_corr,
        )
        regione_idxs = np.random.choice(np.arange(ps.size), idxs.size, p=ps)

        if n_calls < 5 * 10**4:
            _r0_prior = r0_prior_start
        else:
            _r0_prior = r0_prior

        n_calls += 1

        yield (
            deaths[idxs],
            temperatures[idxs],
            S0s[regione_idxs, :],
            I0s[regione_idxs, :],
            R0s[regione_idxs, :],
        ), {
            "ws": np.ones((idxs.size,)),
            "lambda_corr": lambda_corr if n_calls >= delta_corr_init_step else None,
            "c_sum_prior": c_sum_prior,
            "r0_prior": _r0_prior,
            "tau_prior": tau_prior,
            "kappa_prior": kappa_prior,
            "dt_beta_prior": dt_beta_prior,
            "mu_prior": mu_prior,
        }


def beta1_f(step):
    return 0.9


def lr_f(step):
    return power_lr(step=step, lr0=1 * 10**-3, scale=10**4, gamma=0.75)


def display_f(d):
    if (d["step"] + 1) % 10**4 == 0:
        reals = params_to_reals(d["params"])
        kappa, d0, abeta, bbeta, gamma, mu, ah, bh = reals[:8]
        rate0s = reals[8:]

        print_(f"step = {d['step'] + 1}, kappa = {kappa:1.3e}, d0 = {d0:1.2f}")

        betas = beta_t_f(abeta, bbeta, temperatures, densities)

        print_(
            f"    R0 at {temperatures.flatten()[betas.argmax()]:1.2f} °C: "
            + f"{betas.max() / gamma}"
        )
        print_(
            f"    R0 at {temperatures.flatten()[betas.argmin()]:1.2f} °C: "
            + f"{betas.min() / gamma}"
        )

        c = c0 + kappa * pil1_pop0_m * np.exp(-dist_m / d0)
        c_sum = c.sum(axis=0) / pops

        print_(
            f"    Non-commuters account for {(c - c0).sum() / c.sum() * 100:1.3f}% of all flux"
        )
        print_(
            f"    Commuters fraction: max {c_sum.max()} in {regioni[c_sum.argmax()]}, "
            + f" min {c_sum.min()} in {regioni[c_sum.argmin()]}"
        )
        print_(f"    mu = {mu * 10**5:1.3f} deaths/10^5")

        print_()
    return


ro0s = deaths.mean(axis=0) / pops  # (deaths / pops).mean()
params0 = np.array(
    [
        0.1,  # 1.0 + np.random.rand() * 9,
        100.0,
        1 / 100.0,
        np.log(
            (1 - 0.5 ** (1 / (n_steps_a_day * 2.0)))
            * np.exp(temperatures.min() / 100.0)
        ),
        1 - 0.5 ** (1 / (n_steps_a_day * 4.0)),
        1.0 / 10**5,
        1 / 100.0,
        np.log(0.01 * ro0s.mean() * np.exp(-temperatures.max() / 100.0)),
    ]
    + [ro0 for ro0 in ro0s]
)
del ro0s
params0[:] = reals_to_params(params0)


tictoc.tic()
minimize_res = adam(
    params0,
    train_data_generator(adam_data),
    test_data_generator,
    lambda params, data_tuple, ws, lambda_corr, c_sum_prior, r0_prior, tau_prior, kappa_prior, dt_beta_prior, mu_prior: (
        sir(
            params,
            30 * n_steps_a_day,
            data_tuple[0],
            data_tuple[1],
            data_tuple[2],
            data_tuple[3],
            data_tuple[4],
            c0 / n_steps_a_day,
            pil1_pop0_m / n_steps_a_day,
            dist_m,
            pops,
            densities,
            ws,
            lambda_corr,
            corr_array,
            corr_idxs,
            c_sum_prior,
            r0_prior,
            tau_prior,
            kappa_prior,
            dt_beta_prior,
            mu_prior,
            r_nb,
        ),
        jax.grad(
            lambda params: sir(
                params,
                30 * n_steps_a_day,
                data_tuple[0],
                data_tuple[1],
                data_tuple[2],
                data_tuple[3],
                data_tuple[4],
                c0 / n_steps_a_day,
                pil1_pop0_m / n_steps_a_day,
                dist_m,
                pops,
                densities,
                ws,
                lambda_corr,
                corr_array,
                corr_idxs,
                c_sum_prior,
                r0_prior,
                tau_prior,
                kappa_prior,
                dt_beta_prior,
                mu_prior,
                r_nb,
            )
        )(params),
    ),
    n_steps=10**6,
    lr_f=lr_f,
    beta1=beta1_f,
    beta2=0.99,
    eps_stable=1e-8,
    params_history_steps=None,
    mode="adam",
    stop_f=None,
    display_f=display_f,
    internal_data=adam_data,
    clip_values=clip_values,
)
tictoc.toc()

minimize_res_output = print_.get_str()

del print_

## Save the optimisation results on file

In [ ]:
# Save minimize_data to (json) file
d = [
    ("regioni", regioni),
    ("deaths", deaths),
    ("temperatures", temperatures),
    ("c0", c0),
    ("pops", pops),
    ("densities", densities),
    ("pils", pils),
    ("dist_m", dist_m),
    ("pil1_pop0_m", pil1_pop0_m),
    ("n_steps_a_day", n_steps_a_day),
    ("clip_values", clip_values),
    ("ps", ps),
    ("c_sum_prior", c_sum_prior),
    ("r0_prior", r0_prior),
    ("tau_prior", tau_prior),
    ("kappa_prior", kappa_prior),
    ("dt_beta_prior", dt_beta_prior),
    ("lambda_corr", lambda_corr),
    ("delta_corr_init_step", delta_corr_init_step),
    ("n_train_for_corr", n_train_for_corr),
    ("corr_array", corr_array),
    ("corr_idxs", corr_idxs),
    ("test_idxs", test_idxs),
    ("train_idxs", train_idxs),
    ("adam_data", adam_data),
    ("params0", params0),
    ("minimize_res", minimize_res),
    ("minimize_res_output", minimize_res_output),
    ("r_nb", r_nb),
]
d = {c[0]: c[1] for c in d}


def safe_getsource(f):
    try:
        return inspect.getsource(f)
    except:
        print(f"Unable to retrieve source for {f.__name__}")
        return None


d_f = {
    f.__name__: safe_getsource(f)
    for f in globals().values()
    if callable(f)
    and getattr(f, "__name__", None) is not None
    and f.__name__
    in [
        "sig",
        "inv_sig",
        "gammaln_nr",
        "sat_id",
        "lambda_update_step",
        "beta_t_f",
        "lambda_update_month",
        "flat_prior",
        "sir_vec",
        "sir",
        "sir_gen",
        "reals_to_params",
        "params_to_reals",
        "test_data_generator",
        "train_data_generator",
        "beta1_f",
        "lr_f",
        "display_f",
    ]
}
del safe_getsource

d = {**d, **d_f}
del d_f

json_dump(
    d,
    os.path.join(
        data_path,
        "minimize_data_new.json",
    ),
)
del d